<a href="https://colab.research.google.com/github/Sangh0/DeepLearning-Tutorial/blob/main/current_materials/7_overfitting_and_gap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 이번 챕터에서는 over-fitting 해결 방법과 Global Average Pooling에 대해서 살펴볼게요

# Over-fitting (과적합)을 해결하는 방법

- 이번 강의에서는 Over-fitting 문제를 해결하는 방법에 대해서 살펴볼게요
- 먼저, Over-fitting이란 training data에 대해서 성능이 좋은 반면, 그 외의 데이터에는 낮은 성능을 가지는 현상을 의미해요
- 그림으로 살펴볼까요

<img src = "https://pozalabs.github.io/assets/images/Regularization/overfitting.png">

- 우리가 모델을 학습할 때 의도하는 방향성은 데이터 포인트 하나하나 모두 다 맞추는 것이 아니라 경향성만을 파악하는 것이예요
- 즉, 모델의 일반화된 성능을 얻어 어떤 데이터가 들어가든 잘 예측할 수 있도록 학습해야 하죠
- over-fitting 문제를 해결할 수 있는 가장 효과적인 방법은 데이터를 더 많이 확보하는 거예요
- 근데 데이터를 확보하는 것이 어려운 task가 존재해서 over-fitting을 피하기 위한 다양한 방법론들이 연구되었어요

- 1. Early Stopping
    - validation의 loss가 증가하기 시작하는 지점 전까지 model을 저장해 학습을 종료해 over fitting이 일어나지 않도록 설정
    
    <img src = "https://production-media.paperswithcode.com/methods/Screen_Shot_2020-05-28_at_12.59.56_PM_1D7lrVF.png" width=500>

- 2. Regularization or Weight Decay
    - Network의 특정 weight가 너무 커지는 것이 over fitting을 유도할 수 있으므로 weight에 규제를 걸어줌
    - L1 regularization, L2 regularization이 존재
    - L1 regularization은 $Loss = \mathcal{L}(y_{pred}, y_{label}) + \lambda \sum_i^N \vert w_i \vert$
    - L2 regularization은 $Loss = \mathcal{L}(y_{pred}, y_{label}) + \lambda \sum_i^N w_i ^2$
    - L1 regularization
        - 위 식을 미분하면 $\frac{\partial}{\partial w}Loss = \frac{\partial}{\partial w}\mathcal{L} + \lambda \sum_i^N sign(w_i)$이라 할 수 있음
        - 즉, 가중치의 크기가 아니라 오직 부호에만 의존함
        - 또한 중요하지 않은 weight는 0으로 만들어줌 (다른 말로 모델이 결과 예측에 불필요하다고 판단되는 피쳐를 제거하는 기능도 수행)
    - L2 regularization
        - 위 식을 미분하면 $\frac{\partial}{\partial w}Loss = \frac{\partial}{\partial w}\mathcal{L} + \lambda \sum_i^N 2w_i$이라 할 수 있음
        - 즉, weight의 크기에 비례해 큰 값을 가지는 weight는 더 크게 감소되는 기능을 수행
        - 다시 말해, weight의 급격한 변화를 방지하고 모델이 더 안정적으로 학습할 수 있도록 도움을 줌

3. Batch Normalization
    - batch normalization은 feature 값들을 정규화
    - feature 값들이 치우치는 것을 방지하기 위해 평균 0, 분산 1로 만들어줌

    <img src = "https://velog.velcdn.com/images/js03210/post/e01fd3cd-0ae4-4a9f-8701-12a0e30e7056/image.png">

4. Dropout
    - Dropout은 학습 중 일부 노드에 대해서는 동작하지 않도록 설정하는 방법
    - 즉, 일부 노드에 대해 weight 업데이트를 시키지 않음
    - 매 iteration마다 dropout이 random하게 동작해 model ensemble 효과를 가짐
    - 또한 학습 데이터의 모든 샘플에 대해 같은 노드를 사용하지 않아 모델이 특정 샘플에 대해 지나치게 최적화되는 것을 방지함으로써 over-fitting을 막을 수 있음

    <img src = "https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/1-14/04-dropout_overview.png" width=600>

$$a$$

In [ ]:
import numpy as np # 텐서 계산을 위해
import matplotlib.pyplot as plt # 시각화를 위해

import torch # 파이토치 텐서 사용을 위해
import torch.nn as nn # 뉴럴 네트워크 빌드를 위해
import torch.optim as optim # optimizer 사용을 위해
import torchvision.datasets as dsets # torchvision에 내장된 MNIST 데이터셋 다운로드 위해
import torchvision.transforms as transforms # torchvision 전처리를 위해
from torch.utils.data import DataLoader # 딥러닝 학습 데이터로더 구현을 위해

In [ ]:
# Set hyperparameters
Config = {
    'batch_size': 32,
    'learning_rate': 0.01,
    'epochs': 10,
}

In [ ]:
# Load MNIST dataset
train_set = dsets.MNIST(
    root='mnist/',
    train=True,
    transform=transforms.ToTensor(),
    download=True,
)

valid_set = dsets.MNIST(
    root='mnist/',
    train=False,
    transform=transforms.ToTensor(),
    download=True,
)

train_loader = DataLoader(
    dataset=train_set,
    batch_size=Config['batch_size'],
    shuffle=True,
    drop_last=True,
)

valid_loader = DataLoader(
    dataset=valid_set,
    batch_size=Config['batch_size'],
    shuffle=True,
    drop_last=True,
)

In [ ]:
class CNN(nn.Module):

    def __init__(self, in_dim=1, hidden_dim=8, out_dim=10):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            # convolution layer
            nn.Conv2d(in_dim, hidden_dim, kernel_size=3, stride=1, padding=1),
            # batch normalization layer
            nn.BatchNorm2d(hidden_dim),
            # activation layer
            nn.ReLU(),
            # dropout layer
            nn.Dropout(0.1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(hidden_dim, hidden_dim*2, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(hidden_dim*2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(7 * 7 * hidden_dim*2, 100),
            nn.ReLU(),
            nn.Linear(100, out_dim),
        )

    def forward(self, x):
        batch_size = x.size(0)
        x = self.features(x)
        x = x.view(batch_size, -1)
        x = self.classifier(x)
        return x


from torchsummary import summary

summary(CNN(), (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
              ReLU-3            [-1, 8, 28, 28]               0
           Dropout-4            [-1, 8, 28, 28]               0
         MaxPool2d-5            [-1, 8, 14, 14]               0
            Conv2d-6           [-1, 16, 14, 14]           1,168
       BatchNorm2d-7           [-1, 16, 14, 14]              32
              ReLU-8           [-1, 16, 14, 14]               0
           Dropout-9           [-1, 16, 14, 14]               0
        MaxPool2d-10             [-1, 16, 7, 7]               0
           Linear-11                  [-1, 100]          78,500
             ReLU-12                  [-1, 100]               0
           Linear-13                   [-1, 10]           1,010
Total params: 80,806
Trainable params: 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# over-fitting을 유도하기 위해 모델을 복잡하게 빌드
model = CNN(in_dim=1, hidden_dim=16, out_dim=10).to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=Config['learning_rate'])

def cal_accuracy(outputs, labels):
    outputs = torch.argmax(outputs, dim=1)
    correct = (outputs == labels).sum()/len(outputs)
    return correct


train_loss_list, train_acc_list = [], []
valid_loss_list, valid_acc_list = [], []

for epoch in range(Config['epochs']):
    # Training phase ----------------------------------------------------------
    model.train()
    train_loss, train_acc = 0, 0
    for batch, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        acc = cal_accuracy(outputs, labels)
        train_acc += acc.item()
        train_acc_list.append(acc.item())
        loss = loss_func(outputs, labels)
        train_loss += loss.item()
        train_loss_list.append(loss.item())
        loss.backward()
        optimizer.step()


    print(f"# Epoch: {epoch+1}/{Config['epochs']}")
    print(f"# Training phase {'-' * 50}")
    print(f'loss: {train_loss/(batch+1):.3f}, accuracy: {train_acc/(batch+1):.3f}')
    print(f"{'-' * 70}")
    # -------------------------------------------------------------------------

    # Validation phase --------------------------------------------------------
    with torch.no_grad():
        model.eval()
        valid_loss, valid_acc = 0, 0
        for batch, (images, labels) in enumerate(valid_loader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            acc = cal_accuracy(outputs, labels)
            valid_acc += acc.item()
            valid_acc_list.append(acc.item())
            loss = loss_func(outputs, labels)
            valid_loss += loss.item()
            valid_loss_list.append(loss.item())

    print(f"# Validation phase {'-' * 50}")
    print(f'loss: {valid_loss/(batch+1):.3f}, accuracy: {valid_acc/(batch+1):.3f}')
    print(f"{'-' * 70}\n")

# Epoch: 1/10
# Training phase --------------------------------------------------
loss: 0.139, accuracy: 0.958
----------------------------------------------------------------------
# Validation phase --------------------------------------------------
loss: 0.077, accuracy: 0.977
----------------------------------------------------------------------

# Epoch: 2/10
# Training phase --------------------------------------------------
loss: 0.080, accuracy: 0.977
----------------------------------------------------------------------
# Validation phase --------------------------------------------------
loss: 0.096, accuracy: 0.972
----------------------------------------------------------------------

# Epoch: 3/10
# Training phase --------------------------------------------------
loss: 0.079, accuracy: 0.978
----------------------------------------------------------------------
# Validation phase --------------------------------------------------
loss: 0.094, accuracy: 0.974
-------------

# Global Average Pooling

- CNN에서 feature extractor layer에서 classifier로 넘어갈 때 2차원 feature map을 flatten하게 펼쳐 fc (fully connected) layer를 연결해요
- 하지만 이 방식은 문제점이 있어요
    - 1. 공간적 정보를 손실
    - 2. 많은 수의 노드가 나와 학습해야 할 파라미터 수가 증가

- 그래서 제안한 것이 GAP (Global Average Pooling)입니다

<img src = "https://www.researchgate.net/publication/343094775/figure/fig4/AS:915542533763072@1595293758149/Comparison-between-a-fully-connected-layers-in-CNNs-and-b-the-global-average-pooling.png">

- 위 그림에서 왼쪽이 fc layer, 오른쪽이 gap예요
- 이제 fc와 gap를 각각 적용해 파라미터 수의 차이를 알아보죠

In [ ]:
# fc layer 적용한 CNN
class FCcnn(nn.Module):
    def __init__(self):
        super(FCcnn, self).__init__()
        self.features = nn.Sequential(
            # convolution layer
            nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(7 * 7 * 16, 100),
            nn.ReLU(),
            nn.Linear(100, 10),
        )

    def forward(self, x):
        batch_size = x.size(0)
        x = self.features(x)
        x = x.view(batch_size, -1)
        x = self.classifier(x)
        return x

# GAP 적용한 CNN
class GAPcnn(nn.Module):
    def __init__(self):
        super(GAPcnn, self).__init__()
        self.features = nn.Sequential(
            # convolution layer
            nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        # global average pooling 적용
        self.gap = nn.AdaptiveAvgPool2d(output_size=(1, 1))

        self.classifier = nn.Sequential(
            nn.Linear(16, 100),
            nn.ReLU(),
            nn.Linear(100, 10),
        )

    def forward(self, x):
        batch_size = x.size(0)
        x = self.features(x)
        x = self.gap(x)
        x = x.view(batch_size, -1)
        x = self.classifier(x)
        return x

In [ ]:
summary(FCcnn(), (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
              ReLU-2            [-1, 8, 28, 28]               0
         MaxPool2d-3            [-1, 8, 14, 14]               0
            Conv2d-4           [-1, 16, 14, 14]           1,168
              ReLU-5           [-1, 16, 14, 14]               0
         MaxPool2d-6             [-1, 16, 7, 7]               0
            Linear-7                  [-1, 100]          78,500
              ReLU-8                  [-1, 100]               0
            Linear-9                   [-1, 10]           1,010
Total params: 80,758
Trainable params: 80,758
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.16
Params size (MB): 0.31
Estimated Total Size (MB): 0.47
---------------------------------------------

In [ ]:
summary(GAPcnn(), (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
              ReLU-2            [-1, 8, 28, 28]               0
         MaxPool2d-3            [-1, 8, 14, 14]               0
            Conv2d-4           [-1, 16, 14, 14]           1,168
              ReLU-5           [-1, 16, 14, 14]               0
         MaxPool2d-6             [-1, 16, 7, 7]               0
 AdaptiveAvgPool2d-7             [-1, 16, 1, 1]               0
            Linear-8                  [-1, 100]           1,700
              ReLU-9                  [-1, 100]               0
           Linear-10                   [-1, 10]           1,010
Total params: 3,958
Trainable params: 3,958
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.16
Params size (MB): 0.02
Estimated Total